## 1. Importing Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Reading the Data

In [2]:
PROJECT_DIR = r"C:\Users\Rahul Harshawardhan\Desktop\Flight-Price Prediction Sagemaker"
DATA_DIR = "data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,file_name)
    return pd.read_csv(file_path)
    

In [4]:
flights = get_data('flight_price')

In [5]:
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [6]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features
- Column 'Route' and 'Total stops' have missing value each.
- The data types of some features isn't appropriate

# 3. preliminary Analysis

#### 3.1 Check Data Types

In [7]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [8]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [9]:
flights.Date_of_Journey.iloc[6]  # The data type is string which sould be datetime.

'12/03/2019'

#### 3.2 Check For Duplicates

In [10]:
flights.duplicated().sum()

220

In [11]:
(
flights.loc[flights.duplicated(keep= False)].sort_values(['Airline','Date_of_Journey','Source','Destination'])

)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


#### 3.3 Observations

- The type of `Date_of_Journey` should be datatime datatype.
- The type of `Dep_Time & Arrival_Time` should be datatime datatype.
- The data type of `Duration` is mixed. It should be numeric type.
- The `Total_Stops` should be numeric and 'stops' string should be removed from that.
- There are 220 duplicates. These should be removed.

# 4. Detailed Analysis

## Airline

In [12]:
flights['Airline']

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [13]:
flights['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [14]:
# Experimenting with code.
(
flights['Airline']
.str.replace(' Premium economy', '')
.str.replace(' Business', '')
.str.title()
.unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

- Some of the entries have inconsistent/inaccurate values

### Date Of Journey

In [15]:
flights['Date_of_Journey']

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [16]:
pd.to_datetime(flights.Date_of_Journey, dayfirst= True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### source

In [17]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [18]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Route

- This feature is not required, so we will drop this feature.

### Dep_Time

In [19]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [20]:
#Since the departure time is in string data type we can apply some functions of strings. Also we need to check if there are any
# other format type apart from [number:number] format in departure time. For this we will use regular expression.

(
    flights
    .Dep_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [21]:
pd.to_datetime(flights.Dep_Time, format='%H:%M').dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_Time

In [22]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
    .str.split(' ', n=1)
    .str.get(-1).unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [23]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [24]:
(
    flights.Duration
    .loc[lambda ser : ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [25]:
(
    flights.Duration
    .loc[lambda ser : ~ser.str.contains("h")]
    .unique()
)

array(['5m'], dtype=object)

In [26]:
(
    flights.Duration
    .loc[lambda ser : ~ser.str.contains("h")]
    #.unique()
)

6474    5m
Name: Duration, dtype: object

In [27]:
flights.iloc[[6474]] # We will be dropping this row as duration is wrong.

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The observation 6474 has duration of 5min. This is clearly wrong. Will delete this observation

In [28]:
#We have to convert the duration into minutes.


In [29]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand= True) # Expand function converts series to Data Frame.
    .set_axis(["hour", "minutes"], axis = 1)
    .assign
    (
        hour = lambda df_ :
        (

            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
            
        ),
        minutes = lambda df_ : 
        (
            df_
            .minutes
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
        
    )
    .sum(axis =1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

### Total stops

In [30]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [31]:
(
    flights
    .Total_Stops
    .str.replace("non-stop", "0")
    .str.replace(" stops?", "", regex = True) #stops?, question mark says that the last s is optional for checking. means we are #saying pandas to replace stop or stops with "".
    .pipe(lambda ser : pd.to_numeric(ser))                                        
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

# 5. Cleaning Operations

In [32]:
# When we have to create a new column or uodate an existing column we should use Assign function.

In [33]:
# We are creating another function to convert duration to minutes., and this function will be 
                            #integrated with the main clean_data function.
    
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ", expand= True) # Expand function converts series to Data Frame.
        .set_axis(["hour", "minutes"], axis = 1)
        .assign
        (
            hour = lambda df_ :
            (
    
                df_
                .hour
                .str.replace("h", "")
                .astype(int)
                .mul(60)
                
            ),
            minutes = lambda df_ : 
            (
                df_
                .minutes
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
            
        )
        .sum(axis =1)
    ) 

In [34]:
# Important Notes- Pipe function is used to integrate the above 'convert_to_minutes' function in 'clean_data'. Read more about p
# pipe funtion. here series k upar calling pipe, and to pipe i have to give a function and that function should accept series.
# and can return anything.

In [35]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include = 'O').columns
                })
        .rename(columns = str.lower)
        .assign(
            airline = lambda df_ : (
            df_
            .airline
            .str.replace(' Premium economy', '')
            .str.replace(' Business', '')
            .str.title()            
            ),
            date_of_journey = lambda df_ : pd.to_datetime(df_.date_of_journey, dayfirst= True),
            dep_time = lambda df_ : pd.to_datetime(df_.dep_time, format='%H:%M').dt.time,
            arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time, format = 'mixed').dt.time,
            duration = lambda df_ : df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_ :
            (
                df_
                .total_stops
                .str.replace("non-stop", "0")
                .str.replace(" stops?", "", regex = True)
                .pipe(lambda ser : pd.to_numeric(ser))
              
            ),
            additional_info = lambda df_ : df_.additional_info.replace("No info", "No Info")
        
        )
        .drop(columns='route')
    )

In [36]:
flight_cleaned = clean_data(flights)
flight_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


# 6. Split the Data

In [38]:
X = flight_cleaned.drop(columns = 'price')
y = flight_cleaned.price.copy()

In [39]:
X_ , X_test, y_, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size= 0.2, random_state=42)

In [40]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6695, 9) (6695,)
(1674, 9) (1674,)
(2093, 9) (2093,)


# 7Export the subsets

In [43]:
def export_data(X,y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

    X.join(y).to_csv(file_path, index=False)
    return pd.read_csv(file_path).head()

In [44]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087


In [45]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704


In [46]:
export_data(X_test, y_test, "test")


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804
